<a href="https://colab.research.google.com/github/gabikohen/Actividad-de-pipelines-Core-/blob/master/Actividad_de_pipelines_(Core).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [113]:
# Imports
import pandas as pd
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn import set_config
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
set_config(display='diagram')


In [114]:
file = "/content/drive/MyDrive/Colab Notebooks/Coding dojo/Introducion al Machine learning/Practica/Cereal with missing values.xlsx"
data = pd.read_excel(file)
pd.set_option('display.max_columns', None)

In [115]:
data.head(3)

,name,Manufacturer,type,calories per serving,grams of protein,grams of fat,milligrams of sodium,grams of dietary fiber,grams of complex carbohydrates,grams of sugars,milligrams of potassium,vitamins and minerals (% of FDA recommendation),Display shelf,Weight in ounces per one serving,Number of cups in one serving,Rating of cereal
0,Apple Cinnamon Cheerios,General Mills,Cold,110.0,2,2.0,180.0,1.5,10.5,10.0,70,25.0,1,1.00,0.75,29.509541
1,Basic 4,General Mills,Cold,130.0,3,2.0,NaN,2.0,18.0,NaN,100,25.0,3,1.33,0.75,37.038562
2,Cheerios,General Mills,Cold,NaN,6,2.0,290.0,2.0,17.0,1.0,105,25.0,1,1.00,1.25,50.764999


In [116]:
#inspect the data
print(data.info(), '\n')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77 entries, 0 to 76
Data columns (total 16 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   name                                             77 non-null     object 
 1   Manufacturer                                     77 non-null     object 
 2   type                                             68 non-null     object 
 3   calories per serving                             70 non-null     float64
 4   grams of protein                                 77 non-null     int64  
 5   grams of fat                                     69 non-null     float64
 6   milligrams of sodium                             76 non-null     float64
 7   grams of dietary fiber                           77 non-null     float64
 8   grams of complex carbohydrates                   77 non-null     float64
 9   grams of sugars                   

In [117]:
# inspect duplicated

data.duplicated().sum()

0

In [118]:
# Segundo buscamos datos que faltan

print('Hay', data.isna().sum().sum(), 'datos que faltan')

Hay 35 datos que faltan


In [119]:
# Listar por columna
data.isna().sum()

name                                               0
Manufacturer                                       0
type                                               9
calories per serving                               7
grams of protein                                   0
grams of fat                                       8
milligrams of sodium                               1
grams of dietary fiber                             0
grams of complex carbohydrates                     0
grams of sugars                                    9
milligrams of potassium                            0
vitamins and minerals (% of FDA recommendation)    1
Display shelf                                      0
Weight in ounces per one serving                   0
Number of cups in one serving                      0
Rating of cereal                                   0
dtype: int64

Definir la característica (X) y objetivo (y).

In [120]:
#Definir la característica (X) y objetivo (y).

característica = ['Manufacturer',
            'type',
            'grams of fat',
            'grams of sugars',
            'Weight in ounces per one serving']
X = data[característica]
y = data['calories per serving']

In [121]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [122]:
data["Manufacturer"].value_counts()

Kelloggs                       23
General Mills                  22
Post                            9
Quaker Oats                     8
Ralston Purina                  8
Nabisco                         6
American Home Food Products     1
Name: Manufacturer, dtype: int64

Identificar cada característica como numérica, ordinal o nominal. (Entreguen esta pregunta en una celda de texto en un cuaderno de Colab).

> Indented block



Nominales : name, Manufacture,type
Ordinales : No se encontraron datos con las caracteristicas para que sean ordinales
Numericas :grams of protein,grams of fat, milligrams of sodium,grams of dietary fiber,grams of complex carbohydrates,grams of sugars,milligrams of potassium,vitamins and minerals (% of FDA recommendation),Display shelf,
Weight in ounces per one serving,Number of cups in one serving,Rating of cereal

In [123]:
data["type"].value_counts()

Cold    65
Hot      3
Name: type, dtype: int64

In [130]:
# Utilizar transformadores de columnas:select
num_selector = make_column_selector(dtype_include = 'number')
cat_selector = make_column_selector(dtype_include = 'object')

# Utilizar transformadores de columnas:
scaler = StandardScaler() # sirve para estandarizar y tiene una media de 0 y 1
ohe = OneHotEncoder(sparse = False, handle_unknown = 'ignore')


In [138]:
# Imputar algún valor faltante. Utilizar la estrategia “mean” para las columnas numéricas y la estrategia “most_frequent” para las columnas categóricas.
# SimpleImputer sirve para manejar los valores faltantes
mean_imp = SimpleImputer(strategy = 'mean')
freq_imp = SimpleImputer(strategy = 'most_frequent')

# Utilizar pipelines
num_pipeline = make_pipeline(mean_imp, scaler) # make_pipeline Esto facilita la reproducción y el mantenimiento
cat_pipeline = make_pipeline(freq_imp, ohe)



In [133]:
# Hacer tuplas
num_tuple = (num_pipeline, num_selector)
cat_tuple = (cat_pipeline, cat_selector)

# APLICAR
preprocesamiento = make_column_transformer(num_tuple, cat_tuple)

In [139]:
num_tuple

(Pipeline(steps=[('simpleimputer', SimpleImputer()),
                 ('standardscaler', StandardScaler())]),
 <sklearn.compose._column_transformer.make_column_selector at 0x79656d14b400>)

In [134]:
 #Todos los datos de preprocesamiento deben estar contenidos en un único objeto de preprocesamiento.
preprocesamiento.fit(X_train)

# APLICAR
X_train_processed = preprocesamiento.transform(X_train)
X_test_processed = preprocesamiento.transform(X_test)


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [135]:
# evitar la fuga de datos para dejarlos listos para el modelado.
print('Hay',  np.isnan(X_train_processed).sum().sum(), 'datos que faltan en el training.')
print('Hay', np.isnan(X_test_processed).sum().sum(), 'datos que faltan en el set.')


Hay 0 datos que faltan en el training.
Hay 0 datos que faltan en el set.


In [ ]:
# Exp de los datos

print(X_train_processed)
print ('------')
print(X_test_processed)